In [1]:
using Pkg, SymPy
using BenchmarkTools, LinearAlgebra, LazyArrays, BandedMatrices, BlockArrays

Pkg.status()

      Status `~/.julia/environments/v1.7/Project.toml`
  [aae01518] BandedMatrices v0.17.0
  [6e4b80f9] BenchmarkTools v1.3.1
  [8e7c35d0] BlockArrays v0.16.18
  [ce91de38] ColorBitstring v0.1.1
  [7073ff75] IJulia v1.23.3
  [5078a376] LazyArrays v0.22.10
  [a3b82374] MatrixFactorizations v0.9.1
  [91a5bcdd] Plots v1.29.0
  [438e738f] PyCall v1.93.1
  [295af30f] Revise v3.3.3
  [f8ebbe35] SemiseparableMatrices v0.3.3
  [3cc68bcd] SetRounding v0.2.1
  [24249f21] SymPy v1.1.6
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [8dfed614] Test


In [120]:
# l,u = 2,1          # block bandwidths
# N = M = 4          # number of row/column blocks
# cols = rows = 1:N  # block sizes

# BlockBandedMatrix(Zeros(sum(rows),sum(cols)), rows,cols, (l,u)) # creates a block-banded matrix of zeros
# BlockBandedMatrix(Ones(sum(rows),sum(cols)), rows,cols, (l,u)) # creates a block-banded matrix with ones in the non-zero entries
# BlockBandedMatrix(I, rows,cols, (l,u))                          # creates a block-banded  identity matrix

In [2]:
function BackwardSubstitutionV(U, b)
    T = eltype(U)
    n = size(U)[2]
    x = zeros(T, n)
    x[n] = b[n] / U[n, n]
    for i in n-1 : -1 : 1
        sum_ = zero(T)
        for j in i+1 : n
            sum_ += U[i, j] * x[j]
        end
        x[i] = (b[i] - sum_) / U[i, i]
    end
    x
end

function BandedBackwardSubstitutionV(U, b, bw)
    T = eltype(U)
    n = size(U)[2]
    x = zeros(T, n)
    x[n] = b[n] / U[n, n]
    for i in n-1 : -1 : 1
        sum_ = zero(T)
        for j in i+1 : min(n, i+bw)
            sum_ += U[i, j] * x[j]
        end
        x[i] = (b[i] - sum_) / U[i, i]
    end
    x
end

function BandedBackwardSubstitutionM(U, B, bw)
    T = eltype(U)
    n = size(U)[2]
    l = size(B)[2]
    X = zeros(T, n, l)
    for j in l : -1 : 1
        X[:, j] = BandedBackwardSubstitutionV(U, B[:, j], bw)
    end
    X
end

function BackwardSubstitutionM(U, B)
    T = eltype(U)
    n = size(U)[2]
    l = size(B)[2]
    X = zeros(T, n, l)
    for j in l : -1 : 1
        X[:, j] = BackwardSubstitutionV(U, B[:, j])
    end
    X
end

BackwardSubstitutionM (generic function with 1 method)

In [131]:
bw = 1
n = bw*4 + 5

U = big.(BandedMatrix(rand(n, n), (0, bw)))
U += I
T = eltype(U)
n = size(U)[2]

k = n ÷ bw

r = rem(n, bw)

E_k = one(U)[:, n-bw+1:n]
X = BandedBackwardSubstitutionM(U, E_k, bw)

U * X ≈ E_k



Ytk = one(X[n-bw+1:n, :])

# for i in n-3 : -2 : 1
#     # println((i, i+1))
#     Xi = X[i:i+1, :]
#     Di = U[i:i+1, i:i+1]
#     Yti = inv(Xi) * inv(Di)
#     Yt[:, i:i+1] = Yti
#     # display(Di)
# end 

Yt = zero(X')
for i in n : -bw : r+bw
    println(i)
    println((i-bw+1, i))
    Di = U[i-bw+1:i, i-bw+1:i]
    Xi = X[i-bw+1:i, :]
    Yti = inv(Di * Xi)
    # Yti = BackwardSubstitutionM(Xi, one(Xi)) * BackwardSubstitutionM(Di, one(Di))
    # println(Yti * (Di * Xi) ≈ one(Di))
    # println(norm(Yti * (Di * Xi) - one(Di)))
    Yt[:, i-bw+1:i] = Yti
    # print(U[i:i+bw-1, i:i+bw-1])
end


function invR(A)
    A' * inv(A * A')
end
function invL(A)
    inv(A' * A) * A'
end




D0 = U[1:r, 1:r]
println(size(D0))
D0inv = BackwardSubstitutionM(D0, one(D0))
println(size(D0inv))
# print(D0)
X0 = X[1:r, :]
println(size(X0))

println(size(invR(X0)))
X0
D0
Yt0 = invR(D0 * X0)
Yt[:, 1:r] = Yt0

BandedBackwardSubstitutionM(U, one(U), bw) ≈ triu(X * Yt)


9
(9, 9)
8
(8, 8)
7
(7, 7)
6
(6, 6)
5
(5, 5)
4
(4, 4)
3
(3, 3)
2
(2, 2)
1
(1, 1)
(0, 0)
(0, 0)
(0, 1)
(1, 0)


true

In [21]:
function invR(A)
    A' * inv(A * A')
end
function invL(A)
    inv(A' * A) * A'
end

function InvseBlockBiagonalUpper(U, bw)
    n = size(U)[2]
    r = rem(n, bw)
    E_k = one(U)[:, n-bw+1:n]
    # X = BandedBackwardSubstitutionM(U, E_k, bw)
    X = qr(U) \ E_k
    Yt = zero(X')
    for i in n : -bw : r+bw
        Di = U[i-bw+1:i, i-bw+1:i]
        Xi = X[i-bw+1:i, :]
        # Yti = inv(Di * Xi)
        Yti = qr(Di * Xi) \ I
        Yt[:, i-bw+1:i] = Yti
    end
    D0 = U[1:r, 1:r]
    # D0inv = qr(D0) \ I
    # D0inv = inv(D0)
    X0 = X[1:r, :]
    Yt0 = invR(D0 * X0)
    Yt[:, 1:r] = Yt0
    Uinv = triu(X * Yt)
    Uinv
end


bw = 30
n = bw*10 + rand(1:bw-1)

# U = big.(BandedMatrix(rand(n, n), (0, bw)))

# U += 10 * I

A = big.(rand(n, n))
U = triu(A) - triu(A, bw+1) + I

InvseBlockBiagonalUpper(U, bw) * U ≈ I
# norm(InvseBlockBiagonalUpper(U, bw) * U - I)
# norm(InvseBlockBiagonalUpper(U, bw) - inv(U))
# inv(U)

# @time qr(U) \ I
@time inv(U)
@time InvseBlockBiagonalUpper(U, bw);



  1.169242 seconds (62.61 M allocations: 3.266 GiB, 36.60% gc time)
  2.539877 seconds (117.20 M allocations: 6.116 GiB, 36.87% gc time)


In [136]:
a = 21
b = 4
mod(a, b)
# rem(a, b)
a ÷ b
a % b

1

In [35]:
pinv([1 , 2]) * [1 , 2]

1.0

In [36]:
vcat([1, 2], [3, 4])

4-element Vector{Int64}:
 1
 2
 3
 4